In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import pandas as pd
import re
import numpy as np
import base64
import numpy as np
import cv2
import undetected_chromedriver as uc
import argparse
from urllib.parse import urlparse
import io
from PIL import Image
import time
import os
import base64
import numpy as np
import cv2
import undetected_chromedriver as uc
import argparse
from urllib.parse import urlparse
import io
from PIL import Image
import time
import os

In [2]:
# Funciones

def download_image_to_numpy(driver ,url, wait_time=10):
    """
    Downloads an image from a URL that blocks normal requests (like Ripley's CDN)
    and returns it as a numpy array using undetected-chromedriver to bypass protection.
    
    Args:
        url: URL of the image to download
        wait_time: Time to wait for Cloudflare challenges to resolve (seconds)
        
    Returns:
        numpy array of the image
    """

    
    try:

        # Now navigate to the actual image URL
        print(f"Now navigating to the image URL...")
        driver.get(url)
        
        # Additional wait for any challenges
        print(f"Waiting {wait_time} seconds for any Cloudflare challenges...")
        time.sleep(wait_time)
        
        # Check if we're on a Cloudflare challenge page
        page_source = driver.page_source
        if "Just a moment..." in page_source or "Checking your browser" in page_source:
            print("Detected Cloudflare challenge page, waiting for it to resolve...")
            # Wait longer for the challenge to resolve
            time.sleep(wait_time * 2)
        
        # If it's a direct image, the content should be binary, not HTML
        content_type = driver.execute_script("return document.contentType")
        print(f"Content type: {content_type}")
        
        if content_type and "image" in content_type.lower():
            print("Direct image detected, downloading...")
            # For direct image URLs that load in the browser
            get_base64 = """
            const callback = arguments[0];
            let canvas = document.createElement('canvas');
            let img = document.querySelector('img');
            if (!img) {
                // If no img tag (direct image), convert the body to base64
                let reader = new FileReader();
                reader.onloadend = function() {
                    callback(reader.result.split(',')[1]);
                }
                fetch(window.location.href)
                    .then(response => response.blob())
                    .then(blob => reader.readAsDataURL(blob));
            } else {
                // If there's an img tag
                canvas.width = img.naturalWidth;
                canvas.height = img.naturalHeight;
                canvas.getContext('2d').drawImage(img, 0, 0);
                callback(canvas.toDataURL('image/png').split(',')[1]);
            }
            """
        else:
            # For URLs that require fetch
            print("Using fetch to get image content...")
            get_base64 = """
            const url = arguments[0];
            const callback = arguments[1];
            fetch(url, { 
                credentials: 'include',
                headers: {
                    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
                }
            })
            .then(resp => {
                if (!resp.ok) {
                    throw new Error('Network response was not ok: ' + resp.status);
                }
                return resp.blob();
            })
            .then(blob => {
                if (blob.type.startsWith('text/html')) {
                    throw new Error('Received HTML instead of an image');
                }
                const reader = new FileReader();
                reader.onloadend = () => callback(reader.result.split(',')[1]);
                reader.readAsDataURL(blob);
            })
            .catch(error => {
                console.error('Error:', error);
                callback(null);
            });
            """
        
        # Execute the script and get the base64 image data
        base64_str = driver.execute_async_script(get_base64, url) if "image" not in content_type.lower() else driver.execute_async_script(get_base64)
        
        if not base64_str:
            # If we couldn't get the image, take a screenshot as fallback
            print("Failed to get image through JavaScript, trying screenshot of the page...")
            if "image" in content_type.lower():
                screenshot = driver.get_screenshot_as_base64()
                if screenshot:
                    base64_str = screenshot
                    print("Retrieved image via screenshot")
                else:
                    raise RuntimeError("Failed to take screenshot")
            else:
                raise RuntimeError("Failed to download image. Received non-image content.")
        
        # Decode the Base64 to bytes
        img_bytes = base64.b64decode(base64_str)
        
        # Check if we got HTML instead of an image
        if img_bytes[:20].find(b'<!DOCTYPE') != -1 or img_bytes[:20].find(b'<html') != -1:
            # Save the HTML for debugging
            with open("error_page.html", "wb") as f:
                f.write(img_bytes)
            raise RuntimeError("Received HTML instead of an image. Saved to error_page.html for debugging.")
        
        # Intenta primero con OpenCV
        buf = np.frombuffer(img_bytes, dtype=np.uint8)
        img = cv2.imdecode(buf, cv2.IMREAD_UNCHANGED)
        
        # Si OpenCV falla, intenta con PIL/Pillow
        if img is None:
            print("OpenCV failed to decode the image, trying with PIL...")
            pil_img = Image.open(io.BytesIO(img_bytes))
            img = np.array(pil_img)
            
            # Convertir RGBA a BGR si es necesario para OpenCV
            if len(img.shape) == 3 and img.shape[2] == 4:
                # Convertir RGBA a RGB
                rgb_img = pil_img.convert('RGB')
                img = np.array(rgb_img)
                # Convertir RGB a BGR para OpenCV
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            elif len(img.shape) == 3 and img.shape[2] == 3:
                # Convertir RGB a BGR para OpenCV
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                
        if img is None:
            raise RuntimeError("Failed to decode the image with both OpenCV and PIL.")
            
        return img
        
    except Exception as e:
        raise RuntimeError(f"Error downloading image: {str(e)}")
    
    #finally:
        # Always quit the driver to free resources
    #    driver.quit()

In [35]:
import undetected_chromedriver as uc


driver = uc.Chrome(headless=False)  

driver.get("https://simple.ripley.com.pe/moda/especiales/ver-todo-polos?source=search&term=polos&s=mdco")

driver.maximize_window()



In [36]:
descripciones = []
precios_regular = []
precios_oferta = []
imagenes = []
for i in range(1,50):
    try:
        
        descripcion = driver.find_element(By.XPATH, f'/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[3]/div[3]').text ## Descripcion
        if descripcion == None:
           descripciones.append("")
        else:
            descripciones.append(descripcion)

        precio_regular = driver.find_element(By.XPATH, f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[3]/div[4]/div/ul/li[1]").text # Precio regular
        if precio_regular == None:
            precios_regular.append("")
        else:
            precios_regular.append(precio_regular)
        #driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[3]/div[4]/div/ul/li[1]").text 

        precio_oferta = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[3]/div[4]/div/ul/li[2]").text # Precio oferta
        if precio_oferta == None:
            precios_oferta.append("")
        else:
            precios_oferta.append(precio_oferta)
        #driver.find_element(By.XPATH,"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[30]/div/a/div[3]/div[4]/div/ul/li[1]").text
        imagen = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[2]/div[2]/div[1]/img").get_attribute("src") # Imagen
        if imagen == None:
            try:
                imagen = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[2]/div[2]/div[4]/img").get_attribute("src") # Imagen       
                                                
                imagenes.append(imagen)
                if imagen == None:
                    imagen = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[2]/div[2]/div[3]/img").get_attribute("src") # Imagen
                    imagenes.append(imagen)
                    if imagen == None:
                        imagen = driver.find_element(By.XPATH,f"/html/body/div[7]/div[2]/div/div[2]/section/div/div/div[{i}]/div/a/div[2]/div[2]/div[2]/img").get_attribute("src") # Imagen
                        imagenes.append(imagen)
                        
            except:
                pass
        else:
            imagenes.append(imagen)
        
    except:
        pass
                               

In [40]:
len(precios_oferta)
len(precios_regular)

48

In [34]:
len(precio_oferta)

8

In [ ]:

download_image_to_numpy(driver, imagen)

In [30]:

import base64
import numpy as np
import cv2
import undetected_chromedriver as uc

# 1) Inicializa undetected‑chromedriver
driver = uc.Chrome()

# 2) Navega a la página (necesario para pasar la validación)
url = "https://rimage.ripley.com.pe/home.ripley/Attachment/WOP/1/2015346879928/full_image-2015346879928.webp"
driver.get(url)

# 3) Ejecuta un fetch en el contexto del navegador y devuelve el contenido en Base64
get_base64 = """
const url = arguments[0];
const callback = arguments[1];
fetch(url, { credentials: 'include' })
  .then(resp => resp.blob())
  .then(blob => {
    const reader = new FileReader();
    reader.onloadend = () => callback(reader.result.split(',')[1]);
    reader.readAsDataURL(blob);
  })
  .catch(() => callback(null));
"""
base64_str = driver.execute_async_script(get_base64, url)
driver.quit()

if not base64_str:
    raise RuntimeError("No se pudo descargar la imagen desde el contexto del navegador.")

# 4) Decodifica el Base64 a bytes y luego a array con OpenCV
img_bytes = base64.b64decode(base64_str)
buf = np.frombuffer(img_bytes, dtype=np.uint8)
img_bgr = cv2.imdecode(buf, cv2.IMREAD_COLOR)




In [5]:
import undetected_chromedriver as uc
import base64
import numpy as np
import cv2
import undetected_chromedriver as uc
import argparse
from urllib.parse import urlparse
def download_image_to_numpy(url):
    """
    Downloads an image from a URL that blocks normal requests (like Ripley's CDN)
    and returns it as a numpy array using undetected-chromedriver to bypass protection.
    
    Args:
        url: URL of the image to download
        
    Returns:
        numpy array of the image
    """
    # Initialize undetected-chromedriver
    driver = uc.Chrome(headless=True)
    
    try:
        # Navigate to the page
        driver.get(url)
        
        # Execute JS to fetch the image in the browser context
        get_base64 = """
        const url = arguments[0];
        const callback = arguments[1];
        fetch(url, { credentials: 'include' })
          .then(resp => resp.blob())
          .then(blob => {
            const reader = new FileReader();
            reader.onloadend = () => callback(reader.result.split(',')[1]);
            reader.readAsDataURL(blob);
          })
          .catch(error => callback(null));
        """
        
        # Execute the script and get the base64 image data
        base64_str = driver.execute_async_script(get_base64, url)
        
        if not base64_str:
            raise RuntimeError("Failed to download image. The URL might be invalid or blocked.")
        
        # Decode the Base64 to bytes and then to a numpy array with OpenCV
        img_bytes = base64.b64decode(base64_str)
        buf = np.frombuffer(img_bytes, dtype=np.uint8)
        img = cv2.imdecode(buf, cv2.IMREAD_COLOR)
        
        return img
        
    except Exception as e:
        raise RuntimeError(f"Error downloading image: {str(e)}")
    
    finally:
        # Always quit the driver to free resources
        driver.quit()

In [32]:
url = imagen
       
driver = uc.Chrome(headless=True)
    

driver.get(url)

# Execute JS to fetch the image in the browser context
get_base64 = """
const url = arguments[0];
const callback = arguments[1];
fetch(url, { credentials: 'include' })
    .then(resp => resp.blob())
    .then(blob => {
    const reader = new FileReader();
    reader.onloadend = () => callback(reader.result.split(',')[1]);
    reader.readAsDataURL(blob);
    })
    .catch(error => callback(null));
"""

base64_str = driver.execute_async_script(get_base64, url)
img_bytes = base64.b64decode(base64_str)
buf = np.frombuffer(img_bytes, dtype=np.uint8)
cv2.imdecode(buf, cv2.IMREAD_UNCHANGED)

In [ ]:
img_bytes

In [ ]:
import io
from PIL import Image
pil_img = Image.open(io.BytesIO(img_bytes))
img = np.array(pil_img)

In [6]:
download_image_to_numpy()"https://rimage.ripley.com.pe/home.ripley/Attachment/WOP/1/2015346879928/image1-2015346879928.webp"

In [ ]:
import cloudscraper
import numpy as np
import cv2

url = "https://rimage.ripley.com.pe/home.ripley/Attachment/WOP/1/2015346879928/image1-2015346879928.webp"

scraper = cloudscraper.create_scraper(
    browser={
        "browser": "chrome",
        "platform": "windows",
        "mobile": False
    }
)
resp = scraper.get(url)
resp.raise_for_status()

arr = np.frombuffer(resp.content, dtype=np.uint8)
img = cv2.imdecode(arr, cv2.IMREAD_UNCHANGED)
print(img.shape)


In [ ]:
import base64
import numpy as np
import cv2
import undetected_chromedriver as uc
import argparse
from urllib.parse import urlparse
import io
from PIL import Image
import time
import os
def download_image_to_numpy(url, wait_time=10):
    """
    Downloads an image from a URL that blocks normal requests (like Ripley's CDN)
    and returns it as a numpy array using undetected-chromedriver to bypass protection.
    
    Args:
        url: URL of the image to download
        wait_time: Time to wait for Cloudflare challenges to resolve (seconds)
        
    Returns:
        numpy array of the image
    """

    
    driver = uc.Chrome( headless=False)  # Set headless=False to see the browser
    
    try:
        # First navigate to the site's homepage to set cookies
        base_url = urlparse(url)
        homepage = f"{base_url.scheme}://{base_url.netloc}"
        print(f"First navigating to homepage: {homepage}")
        driver.get("https://simple.ripley.com.pe/moda/especiales/ver-todo-polos?source=search&term=polos&s=mdco")
        
        # Wait for the homepage to load and potential Cloudflare checks to pass
        print(f"Waiting {wait_time} seconds for initial page load...")
        time.sleep(wait_time)
        
        # Now navigate to the actual image URL
        print(f"Now navigating to the image URL...")
        driver.get(url)
        
        # Additional wait for any challenges
        print(f"Waiting {wait_time} seconds for any Cloudflare challenges...")
        time.sleep(wait_time)
        
        # Check if we're on a Cloudflare challenge page
        page_source = driver.page_source
        if "Just a moment..." in page_source or "Checking your browser" in page_source:
            print("Detected Cloudflare challenge page, waiting for it to resolve...")
            # Wait longer for the challenge to resolve
            time.sleep(wait_time * 2)
        
        # If it's a direct image, the content should be binary, not HTML
        content_type = driver.execute_script("return document.contentType")
        print(f"Content type: {content_type}")
        
        if content_type and "image" in content_type.lower():
            print("Direct image detected, downloading...")
            # For direct image URLs that load in the browser
            get_base64 = """
            const callback = arguments[0];
            let canvas = document.createElement('canvas');
            let img = document.querySelector('img');
            if (!img) {
                // If no img tag (direct image), convert the body to base64
                let reader = new FileReader();
                reader.onloadend = function() {
                    callback(reader.result.split(',')[1]);
                }
                fetch(window.location.href)
                    .then(response => response.blob())
                    .then(blob => reader.readAsDataURL(blob));
            } else {
                // If there's an img tag
                canvas.width = img.naturalWidth;
                canvas.height = img.naturalHeight;
                canvas.getContext('2d').drawImage(img, 0, 0);
                callback(canvas.toDataURL('image/png').split(',')[1]);
            }
            """
        else:
            # For URLs that require fetch
            print("Using fetch to get image content...")
            get_base64 = """
            const url = arguments[0];
            const callback = arguments[1];
            fetch(url, { 
                credentials: 'include',
                headers: {
                    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
                }
            })
            .then(resp => {
                if (!resp.ok) {
                    throw new Error('Network response was not ok: ' + resp.status);
                }
                return resp.blob();
            })
            .then(blob => {
                if (blob.type.startsWith('text/html')) {
                    throw new Error('Received HTML instead of an image');
                }
                const reader = new FileReader();
                reader.onloadend = () => callback(reader.result.split(',')[1]);
                reader.readAsDataURL(blob);
            })
            .catch(error => {
                console.error('Error:', error);
                callback(null);
            });
            """
        
        # Execute the script and get the base64 image data
        base64_str = driver.execute_async_script(get_base64, url) if "image" not in content_type.lower() else driver.execute_async_script(get_base64)
        
        if not base64_str:
            # If we couldn't get the image, take a screenshot as fallback
            print("Failed to get image through JavaScript, trying screenshot of the page...")
            if "image" in content_type.lower():
                screenshot = driver.get_screenshot_as_base64()
                if screenshot:
                    base64_str = screenshot
                    print("Retrieved image via screenshot")
                else:
                    raise RuntimeError("Failed to take screenshot")
            else:
                raise RuntimeError("Failed to download image. Received non-image content.")
        
        # Decode the Base64 to bytes
        img_bytes = base64.b64decode(base64_str)
        
        # Check if we got HTML instead of an image
        if img_bytes[:20].find(b'<!DOCTYPE') != -1 or img_bytes[:20].find(b'<html') != -1:
            # Save the HTML for debugging
            with open("error_page.html", "wb") as f:
                f.write(img_bytes)
            raise RuntimeError("Received HTML instead of an image. Saved to error_page.html for debugging.")
        
        # Intenta primero con OpenCV
        buf = np.frombuffer(img_bytes, dtype=np.uint8)
        img = cv2.imdecode(buf, cv2.IMREAD_UNCHANGED)
        
        # Si OpenCV falla, intenta con PIL/Pillow
        if img is None:
            print("OpenCV failed to decode the image, trying with PIL...")
            pil_img = Image.open(io.BytesIO(img_bytes))
            img = np.array(pil_img)
            
            # Convertir RGBA a BGR si es necesario para OpenCV
            if len(img.shape) == 3 and img.shape[2] == 4:
                # Convertir RGBA a RGB
                rgb_img = pil_img.convert('RGB')
                img = np.array(rgb_img)
                # Convertir RGB a BGR para OpenCV
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            elif len(img.shape) == 3 and img.shape[2] == 3:
                # Convertir RGB a BGR para OpenCV
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                
        if img is None:
            raise RuntimeError("Failed to decode the image with both OpenCV and PIL.")
            
        return img
        
    except Exception as e:
        raise RuntimeError(f"Error downloading image: {str(e)}")
    
    finally:
        # Always quit the driver to free resources
        driver.quit()

ejemplo_imagen = download_image_to_numpy('https://rimage.ripley.com.pe/home.ripley/Attachment/WOP/1/2015346879928/full_image-2015346879928.webp')